### Importing libraries

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import TensorBoard
from time import time
import numpy as np

# For using core package that located in the two upper folder.
import sys
sys.path.append('../../')

from core.json_importer import parse_json_file, parse_all_files
from core.filters import remove_unpressed_frames, remove_both_standing_frames, remove_same_consecutive_actions, remove_recov_frames
from core.actions import one_hot_encode, decode
from core.preproccessing import Normalizer
from core.helpers import write_file

Using TensorFlow backend.


In [2]:
# Open file
game_data = parse_all_files("data/utalvsthunder")

#batch_size is 1000, utalvsthunder and bcpvsthunder data folders can be found at drive

#### Remove unneeded frames

In [3]:
l1 = game_data.filter(remove_both_standing_frames)
l3 = game_data.filter(remove_recov_frames)
# l4 = game_data.filter(remove_same_consecutive_actions)

print("Removed {} frames that both players are standing".format(l1))
print("Removed {} frames that are recov frames".format(l3))
# print("Removed {} frames that are remove_same_consecutive_actions".format(l4))

print("\n{} frames removed.".format(l1 + l3))
print("{} frames left.".format(len(game_data)))

Removed 323 frames that both players are standing
Removed 1551 frames that are recov frames

1874 frames removed.
8926 frames left.


#### Encoding and normalization

In [4]:

def normalize_and_save(data, file_name):
    p1_hp_normalizer = Normalizer()
    p1_normalized_hp = p1_hp_normalizer.normalize(data)
    p1_hp_normalizer.save("out/" + file_name)
    return p1_normalized_hp

## Pre process data
processed_data = []

# Create one hot encoding for actions (For input and labels)
p1_one_hot_encoded_actions = one_hot_encode(game_data.get_column("P1-action"))
labels = one_hot_encode(game_data.get_column("P2-action"))

# Normalize uncategorized features
p1_normalized_hp = normalize_and_save(game_data.get_column("P1-hp"), "p1_hp_norm.save")
p2_normalized_hp = normalize_and_save(game_data.get_column("P2-hp"), "p2_hp_norm.save")
normalized_x_distance = normalize_and_save([frame["P1-x"] - frame["P2-x"] for frame in game_data], "x_norm.save")
normalized_y_distance = normalize_and_save([frame["P1-y"] - frame["P2-y"] for frame in game_data], "y_norm.save")
normalized_xp1_distance = normalize_and_save([frame["P1-x"] for frame in game_data], "xp1_norm.save")
normalized_xp2_distance = normalize_and_save([frame["P2-x"] for frame in game_data], "xp2_norm.save")
normalized_yp1_distance = normalize_and_save([frame["P1-y"] for frame in game_data], "yp1_norm.save")
normalized_yp2_distance = normalize_and_save([frame["P2-y"] for frame in game_data], "yp2_norm.save")

for index in range(len(game_data)):    
    processed_row = []
    processed_row.extend(p1_one_hot_encoded_actions[index])
    processed_row.extend(p1_normalized_hp[index]) 
    processed_row.extend(p2_normalized_hp[index])
    processed_row.extend(normalized_x_distance[index])
    processed_row.extend(normalized_y_distance[index])
    processed_row.extend(normalized_xp1_distance[index])
    processed_row.extend(normalized_xp2_distance[index])
    processed_row.extend(normalized_yp1_distance[index])
    processed_row.extend(normalized_yp2_distance[index])
    processed_data.append(processed_row)
processed_data = np.array(processed_data)
labels = np.array(labels)

### Example data

In [5]:
EXAMPLE_ROW = 10

print("There are %d frames in dataset." % len(processed_data))
print("After pre processing the shape of our dataset is %s" % str(processed_data.shape))
print("\nOne example in index %d." % EXAMPLE_ROW)

row = processed_data[EXAMPLE_ROW]
print("\tProcessed Frame:" )
print("\t\tP1 Action(one-hot):\t%s" % row[0: 55])
print("\t\tP1 Hp(norm.):\t\t%s" % row[55])
print("\t\tP2 Hp(norm.):\t\t%s" % row[56])
print("\t\tX Dist(norm.):\t\t%s" % row[57])
print("\t\tY Dist(norm.):\t\t%s" % row[58])
print("\t\tXP1 (norm.):\t\t%s" % row[59])
print("\t\tXP2 (norm.):\t\t%s" % row[60])
print("\t\tYP1 (norm.):\t\t%s" % row[61])
print("\t\tYP2 (norm.):\t\t%s" % row[62])

print("\t\tLabel:\t\t\t%s" % labels[EXAMPLE_ROW])

There are 8926 frames in dataset.
After pre processing the shape of our dataset is (8926, 63)

One example in index 10.
	Processed Frame:
		P1 Action(one-hot):	[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
		P1 Hp(norm.):		0.0
		P2 Hp(norm.):		0.0
		X Dist(norm.):		0.03940886699507389
		Y Dist(norm.):		0.07291666666666667
		XP1 (norm.):		0.35000000000000003
		XP2 (norm.):		0.36921151439299127
		YP1 (norm.):		0.9999999999999999
		YP2 (norm.):		0.8942598187311178
		Label:			[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### Neural Network Design

Our neural network has two hidden layers in this test. They has 12 and 8 neurons respectively.

In [6]:
# Constants
INPUT_LAYER_SIZE = processed_data.shape[1]
OUTPUT_LAYER_SIZE = labels.shape[1]

In [7]:
model = Sequential()
model.add(Dense(12, input_dim=INPUT_LAYER_SIZE, activation='relu'))
model.add(Dense(56, activation='relu'))
model.add(Dense(OUTPUT_LAYER_SIZE, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# Logging for tensorboard
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

In [9]:
model.fit(processed_data, labels, epochs=150, batch_size=1000, callbacks=[tensorboard])

Epoch 1/150
8926/8926 [==============================] - 0s 23us/step - loss: 0.6884 - acc: 0.5571
Epoch 2/150
8926/8926 [==============================] - 0s 6us/step - loss: 0.6590 - acc: 0.6776
Epoch 3/150
8926/8926 [==============================] - 0s 6us/step - loss: 0.6208 - acc: 0.7967
Epoch 4/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.5666 - acc: 0.8934
Epoch 5/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.4907 - acc: 0.9490
Epoch 6/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.3926 - acc: 0.9802
Epoch 7/150
8926/8926 [==============================] - 0s 6us/step - loss: 0.2833 - acc: 0.9840
Epoch 8/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.1854 - acc: 0.9840
Epoch 9/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.1179 - acc: 0.9846
Epoch 10/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.0807 - acc: 0.9847
Epoch 11/150
8926/

8926/8926 [==============================] - 0s 5us/step - loss: 0.0261 - acc: 0.9911
Epoch 84/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.0260 - acc: 0.9911
Epoch 85/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.0260 - acc: 0.9912
Epoch 86/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.0259 - acc: 0.9912
Epoch 87/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.0258 - acc: 0.9912
Epoch 88/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.0258 - acc: 0.9912
Epoch 89/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.0257 - acc: 0.9912
Epoch 90/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.0257 - acc: 0.9912
Epoch 91/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.0256 - acc: 0.9912
Epoch 92/150
8926/8926 [==============================] - 0s 5us/step - loss: 0.0255 - acc: 0.9912
Epoch 93/150
8926/8926 

In [10]:
scores = model.evaluate(processed_data, labels)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

8926/8926 [==============================] - 0s 16us/step

acc: 99.21%


### Save model

In [11]:
model.save("out/model.h5")
write_file("out/config.json", model.to_json())

In [12]:
game_data.rounds[0][0][game_data.columns.index("P2-action")]

'FORWARD_WALK'